# 서브 워드 토크 나이저

이번 튜토리얼에서는 `text.BertTokenizer`를 이용해 데이터 세트에서 subword 어휘를 생성하고 구축하는 방법을 보여줍니다.

`tensorflow_text` 패키지에는 토크나이저들이 구현되어 있습니다. 그 중 3가지는 아래와 같습니다.

+ `text_BertTokenizer` : 높은 수준의 인터페이스를 갖고 있습니다. BERT의 토큰 분할 알고리즘과 `WordPieceTokenizer`가 포함되어 있습니다. 문장을 입력받아 토큰 ID를 반환합니다.
+ `text.WordpieceTokenizer` : 낮은 수준의 인터페이스를 갖습니다. WordPiece 알고리즘만을 구현했습니다. 텍스트를 호출하기 전에 표준화 작업과 단어를 분할하는 작업이 선행되어야 합니다. 단어를 입력받아 토큰 ID를 반환합니다.
+ `text.SentencepieceTokenizer` : 사전 훈련된 문장 모델이 필요합니다. 이 모델을 이용해서 문장을 입력으로 받을 수 있습니다.

본 튜토리얼은 하향식 방식으로 Wordpiece 어휘를 빌드합니다. 일본어, 중국어, 한국어에는 동작하지 않는 프로세스입니다. 이러한 언어를 토큰화하려면 `text.SentencepieceTokenizer` 또는 [text.UnicodeCharTokenizer](https://www.tensorflow.org/text/api_docs/python/text/UnicodeCharTokenizer)를 이용해야 합니다.

In [1]:
!pip install -q -U tensorflow-text

In [2]:
!pip install -q tensorflow_datasets

In [3]:
import collections
import os, pathlib, re, string, sys, tempfile, time
import numpy as np
import matplotlib.pyplot as plt
import tensorflow_datasets as tfds
import tensorflow_text as text
import tensorflow as tf

In [4]:
tf.get_logger().setLevel('ERROR')
pwd = pathlib.Path.cwd()

## 데이터셋 다운로드

`tensorflow_datasets`에서 포르투갈어 / 영어 번역 데이터셋을 가져옵니다.

In [5]:
examples, metadata = tfds.load('ted_hrlr_translate/pt_to_en', with_info=True,
                               as_supervised=True)
train_examples, val_examples = examples['train'], examples['validation']

이 데이터셋은 포르투갈어 / 영어 문장 쌍을 생성합니다.

In [6]:
for pt, en in train_examples.take(1):
    print("Portuguese: ", pt.numpy().decode('utf-8'))
    print("English: ", pt.numpy().decode('utf-8'))

Portuguese:  e quando melhoramos a procura , tiramos a única vantagem da impressão , que é a serendipidade .
English:  e quando melhoramos a procura , tiramos a única vantagem da impressão , que é a serendipidade .


데이터셋은 아래와 같은 사항에 유의해야 합니다.

+ 소문자로 이루어져있습니다.
+ 구두점 주위에 공백이 있습니다.
+ 그렇다고 이 데이터셋의 정규화 여부가 명확하지 않습니다.

In [7]:
train_en = train_examples.map(lambda pt, en: en)
train_pt = train_examples.map(lambda pt, en: pt)

## 어휘 생성

이번 섹션에서는 데이터셋에서 vocabulary를 생성합니다. vocabulary 파일은 이미 있고 생성 코드가 `tensorflow_text` pip 패키지에 포함되어 있습니다. 자동으로 가져 오지 않기 때문에 수동으로 가져와야 합니다.


In [8]:
from tensorflow_text.tools.wordpiece_vocab import bert_vocab_from_dataset as bert_vocab

`bert_vocab.bert_vocab_from_dataset` 함수는 vocabulary를 생성합니다.

[Wordpiece 알고리즘](#algorithm) 설명


In [9]:
bert_tokenizer_params = dict(lower_case=True)  # 정규화(소문자화)
reserved_tokens=["[PAD]", "[UNK]", "[START]", "[END]"]

bert_vocab_args = dict(
    vocab_size = 8000,
    reserved_tokens=reserved_tokens,
    bert_tokenizer_params=bert_tokenizer_params,
    learn_params={}
)

In [10]:
%%time
pt_vocab = bert_vocab.bert_vocab_from_dataset(
    train_pt.batch(1000).prefetch(2),
    **bert_vocab_args
)

CPU times: user 2min 18s, sys: 3.12 s, total: 2min 21s
Wall time: 2min 19s


In [11]:
print(pt_vocab[:10])
print(pt_vocab[100:110])
print(pt_vocab[1000:1010])
print(pt_vocab[-10:])

['[PAD]', '[UNK]', '[START]', '[END]', '!', '#', '$', '%', '&', "'"]
['no', 'por', 'mais', 'na', 'eu', 'esta', 'muito', 'isso', 'isto', 'sao']
['90', 'desse', 'efeito', 'malaria', 'normalmente', 'palestra', 'recentemente', '##nca', 'bons', 'chave']
['##–', '##—', '##‘', '##’', '##“', '##”', '##⁄', '##€', '##♪', '##♫']


어휘 파일을 작성합니다.

In [13]:
def write_vocab_file(filepath, vocab):
  with open(filepath, 'w') as f:
    for token in vocab:
      print(token, file=f)

In [14]:
write_vocab_file('pt_vocab.txt', pt_vocab)

In [15]:
%%time
en_vocab = bert_vocab.bert_vocab_from_dataset(
    train_en.batch(1000).prefetch(2),
    **bert_vocab_args
)

CPU times: user 1min 35s, sys: 2.98 s, total: 1min 38s
Wall time: 1min 36s
